# Windows Registry Persistence - COM key linking
Detects COM object hijacking via TreatAs subkey

## Rule Content
```
- title: Windows Registry Persistence - COM key linking
  id: 9b0f8a61-91b2-464f-aceb-0527e0a45020
  status: experimental
  description: Detects COM object hijacking via TreatAs subkey
  references:
  - https://bohops.com/2018/08/18/abusing-the-com-registry-structure-part-2-loading-techniques-for-evasion-and-persistence/
  author: Kutepov Anton, oscd.community
  date: 2019/10/23
  modified: 2019/11/07
  tags:
  - attack.persistence
  - attack.t1122
  logsource:
    product: windows
    service: sysmon
    category: null
  detection:
    selection:
      EventID: 12
      TargetObject|startswith: HKU\
      TargetObject|contains: _Classes\CLSID\
      TargetObject|endswith: \TreatAs
    condition: selection
  falsepositives:
  - Maybe some system utilities in rare cases use linking keys for backward compability
  level: medium

```

## Querying Elasticsearch

### Import Libraries

In [ ]:
from elasticsearch import Elasticsearch
from elasticsearch_dsl import Search
import pandas as pd

### Initialize Elasticsearch client

In [ ]:
es = Elasticsearch(['http://helk-elasticsearch:9200'])
searchContext = Search(using=es, index='logs-endpoint-winevent-sysmon-*', doc_type='doc')

### Run Elasticsearch Query

In [ ]:
s = searchContext.query('query_string', query='(event_id:"12" AND registry_key_path:"HKU\*" AND registry_key_path.keyword:*_Classes\\CLSID\* AND registry_key_path.keyword:*\\TreatAs)')
response = s.execute()
if response.success():
    df = pd.DataFrame((d.to_dict() for d in s.scan()))

### Show Results

In [ ]:
df.head()